In [88]:
# import pandas as pd
import numpy as np
import random
import math

Suppose the triangle has Pythagorean triple $(a, b, c)$ as its sides, with $a < b < c$. Given a limit $L$, we are looking for triangles such that $a + b + c < L$ such that $(b-a)^2 \, | \, c^2$. That is, $\exists k \in \mathbb{Z}$ such that
$$
(b-a)^2 \, | \, c^2 \iff k(b-a)^2 = c^2 \iff k(a^2 - 2ab + b^2) = c^2 \iff k(c^2 - 2ab) = c^2 \iff \frac{k - 1}{k} c^2 = 2ab.
$$

From the initial setup, consider $k(b-a)^2 = c^2$. Since the right hand side is a perfect square and $c$ is an integer, the left hand side must also be a perfect square. That is $k = d^2$ for some $d \in \mathbb{N}$, since $(b-a)^2$ is already a perfect square. So we simply get $d(b - a) = c \iff b-a \, | \; c$.

For any multiple of primitive Pythagorean triple $(a,b,c)$ and integer $k \in \mathbb{Z}$, $(na, nb, nc)$ for some $n \in \mathbb{N}$, we have
$$
k(b-a)^2 = c^2 \iff kn^2(b - a)^2 = n^2c^2 \iff k(nb - na)^2 = (nc^2).
$$

This means we only need to check if a primitive triple satisfies the property for some integer $k$, and then we know all its multiples by simply summing $\lfloor \frac{L}{a + b + c} \rfloor$. Additionally, if a primitive triple does not satisfy the property, we know that none of the multiples will satisfy it either.



Since we know exactly one of $a$ or $b$ is even, that implies exactly one of $a^2$ or $b^2$ is even, so we know $c^2$ must be odd (and $c$ must be odd). But if we look at $\frac{k - 1}{k} c^2 = 2ab$, the right hand side is an even integer! This means the only valid values of $k$ to check are when $\frac{k-1}{k}$ is an even integer (since $c^2$ is an odd integer).
This tells us two things:
1. $k-1 \, \!\not| \; k$ for any $k \in \mathbb{Z}$ so that means $k \, | \, c^2$. In particular since $c^2$ is odd, $k$ is odd, and $\frac{c^2}{k}$ is also odd, since $c^2$ cannot have even factors.
2. Since $k$ is odd, $k-1$ is even, and hence we can divide out the $2$ on both sides. Thus we can rewrite as
$$
\frac{k-1}{2k} c^2 = ab \implies \frac{c^2}{k} \, \Bigg| \, ab
$$
and we only need to check for $k$ where $k$ is a factor of $c^2$. But remember $k = d^2$! So in summary, we know $k$ is a factor of $c$ that must be an odd perfect square.

In [84]:
# cached version of gcd
cache = {}
def gcd(a,b):
    if not a:
        return b

    if (a,b) not in cache:
        cache[(a,b)] = gcd(b % a, a)
    
    return cache[(a,b)]

# Euclid's formula
def euclid_pythagorean(max_perimeter):
    if max_perimeter < 12:
        return []

    triples = []
    to_check = [(2, 1)]
    seen = set()
    while to_check:
        m,n = to_check.pop(0)
        if (m,n) in seen: continue

        seen.add((m,n))

        # require gcd = 1 for primitive triples
        # require one exactly one of m or n to be even
        if gcd(m,n) == 1 and (m + n) % 2 == 1:
            a,b,c = m*m - n*n, 2*m*n, m*m + n*n
            
            if a+b+c >= max_perimeter: continue

            triples.append((min(a,b),max(a,b),c))

        if m > n + 1:
            to_check.append((m, n+1))
        to_check.append((m+1, n))

    return triples

trips = euclid_pythagorean(10**8)

In [85]:
len(trips)

7023027

In [87]:
sorted(trips, key = lambda x: np.sum(x))

[(3, 4, 5),
 (5, 12, 13),
 (8, 15, 17),
 (7, 24, 25),
 (20, 21, 29),
 (12, 35, 37),
 (9, 40, 41),
 (28, 45, 53),
 (11, 60, 61),
 (16, 63, 65),
 (33, 56, 65),
 (48, 55, 73),
 (13, 84, 85),
 (36, 77, 85),
 (39, 80, 89),
 (20, 99, 101),
 (65, 72, 97),
 (15, 112, 113),
 (60, 91, 109),
 (44, 117, 125),
 (17, 144, 145),
 (24, 143, 145),
 (88, 105, 137),
 (51, 140, 149),
 (85, 132, 157),
 (19, 180, 181),
 (52, 165, 173),
 (119, 120, 169),
 (57, 176, 185),
 (28, 195, 197),
 (104, 153, 185),
 (95, 168, 193),
 (21, 220, 221),
 (84, 187, 205),
 (133, 156, 205),
 (60, 221, 229),
 (140, 171, 221),
 (32, 255, 257),
 (105, 208, 233),
 (23, 264, 265),
 (120, 209, 241),
 (69, 260, 269),
 (96, 247, 265),
 (115, 252, 277),
 (68, 285, 293),
 (25, 312, 313),
 (160, 231, 281),
 (36, 323, 325),
 (161, 240, 289),
 (75, 308, 317),
 (136, 273, 305),
 (207, 224, 305),
 (27, 364, 365),
 (204, 253, 325),
 (76, 357, 365),
 (175, 288, 337),
 (180, 299, 349),
 (40, 399, 401),
 (225, 272, 353),
 (135, 352, 377),
 (29,

In [89]:
def miller_rabin(n, k = 10):
    if n <= 3:
        return n == 2 or n == 3

    if n % 2 == 0:
        return False

    r, s = 0, n - 1
    while s % 2 == 0:
        r += 1
        s //= 2
    for _ in range(k):
        a = random.randrange(2, n - 1)
        x = pow(a, s, n)
        if x == 1 or x == n - 1:
            continue
        for _ in range(r - 1):
            x = pow(x, 2, n)
            if x == n - 1:
                break
        else:
            return False
    return True

In [91]:
count = 0

for i, (a,b,c) in enumerate(trips):
    if i % 10**6 == 0:
        print(i)

    if miller_rabin(c):
        if b-a == 1:
            print('Diff is 1:', a, b, c)
            count += 1
    elif c % (b-a) == 0:
        print('Is divisible:', a, b, c)
        count += 1

0
Diff is 1: 3 4 5
Diff is 1: 20 21 29
Is divisible: 119 120 169
Is divisible: 696 697 985
Diff is 1: 4059 4060 5741
Diff is 1: 23660 23661 33461
Is divisible: 137903 137904 195025
Is divisible: 803760 803761 1136689


KeyboardInterrupt: 